In [8]:

!pip install pyswarms

# Importing necessary libraries
import numpy as np
import pandas as pd
from pyswarms.single.global_best import GlobalBestPSO


# --------------- Define Cost Parameters ---------------

# Full-time hourly rates
full_time_costs = np.array([30, 35, 40, 45, 50])

# Overtime hourly rates (1.5 times full-time rates)
overtime_costs = 1.5 * full_time_costs

# Part-time hourly rates
part_time_costs = np.array([25, 27, 29, 31, 33])

# Combine all cost rates into one cost vector for the 15 decision variables
cost_vector = np.concatenate([full_time_costs, overtime_costs, part_time_costs])


# --------------- Function to Calculate Total Labor Expenses ---------------

def calculate_labor_expenses(x):
    penalty_values = []  # List to store penalty costs for each solution

    for xi in x:
        xi = np.clip(xi, 0, None)  # Ensure non-negative working hours
        
        # Calculate peak (full-time + overtime) and non-peak (part-time) hours
        peak_hours = np.sum(xi[:5]) + np.sum(xi[5:10])
        non_peak_hours = np.sum(xi[10:15])

        penalty = 0
        # Apply penalty if peak hours are less than 4
        if peak_hours < 4:
            penalty += (4 - peak_hours) * 1000  
        # Apply penalty if non-peak (part-time) hours are less than 2
        if non_peak_hours < 2:
            penalty += (2 - non_peak_hours) * 1000  

        # Calculate total cost including penalties
        total_cost = np.sum(cost_vector * xi)  
        penalty_values.append(total_cost + penalty)

    return np.array(penalty_values)


# --------------- Set Bounds and PSO Parameters ---------------

lower_bound = np.zeros(15)  # Minimum working hours for each category
upper_bound = np.ones(15) * 8  # Maximum working hours (8 hours per category)
pso_options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7}  # PSO configuration parameters


# --------------- Run Particle Swarm Optimization (PSO) ---------------

# Initialize PSO optimizer
optimizer = GlobalBestPSO(n_particles=30, dimensions=15, options=pso_options, bounds=(lower_bound, upper_bound))

# Optimize to find the best cost and position
best_cost, optimal_position = optimizer.optimize(calculate_labor_expenses, iters=100)


# --------------- Categorize and Display Results ---------------

# Define the categories for full-time, overtime, and part-time
categories = ['Full-time (x_i)', 'Overtime (x_io)', 'Part-time (x_jp)']

# Combine all hourly rates into a single array
rates = np.concatenate([full_time_costs, overtime_costs, part_time_costs])

# Create a DataFrame to display the results of the optimization
df = pd.DataFrame({
    'Category': [categories[i // 5] for i in range(15)],  # Categorize the 15 decision variables
    'Hourly Rate': rates,
    'Hours Worked': optimal_position,
    'Cost': rates * optimal_position  # Calculate the total cost based on hours worked
})


# --------------- Calculate Peak and Non-Peak Hours ---------------

# Sum of peak hours (full-time + overtime)
total_peak_hours = np.sum(optimal_position[:10])

# Sum of non-peak (part-time) hours
total_non_peak_hours = np.sum(optimal_position[10:15])


# --------------- Display Output Results ---------------

# Print total labor cost and hour summaries
print(f"Total Labor Cost: {best_cost:.2f} SAR")
print(f"Total Peak Hours: {total_peak_hours:.2f} hours (Minimum required: 4 hours)")
print(f"Total Non-Peak Part-Time Hours: {total_non_peak_hours:.2f} hours (Minimum required: 2 hours)")

# Display the detailed DataFrame with cost breakdown
df

2025-04-20 22:00:33,666 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|100/100, best_cost=1.13e+3
2025-04-20 22:00:34,388 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1130.2276194333851, best pos: [3.23942892 0.80831039 2.09848587 2.11768756 0.96962727 1.05425542
 1.00797525 1.48159077 2.56408224 2.97030727 3.12997083 0.11715561
 0.46343706 2.64149729 0.46043483]


Total Labor Cost: 1130.23 SAR
Total Peak Hours: 18.31 hours (Minimum required: 4 hours)
Total Non-Peak Part-Time Hours: 6.81 hours (Minimum required: 2 hours)


,Category,Hourly Rate,Hours Worked,Cost
0,Full-time (x_i),30.0,3.239429,97.182868
1,Full-time (x_i),35.0,0.808310,28.290864
2,Full-time (x_i),40.0,2.098486,83.939435
3,Full-time (x_i),45.0,2.117688,95.295940
4,Full-time (x_i),50.0,0.969627,48.481363
5,Overtime (x_io),45.0,1.054255,47.441494
6,Overtime (x_io),52.5,1.007975,52.918701
7,Overtime (x_io),60.0,1.481591,88.895446
8,Overtime (x_io),67.5,2.564082,173.075552
9,Overtime (x_io),75.0,2.970307,222.773045
